# Ham Sandwich Tree

El Ham Sandwich Tree es una estructura de datos que tiene como objetivo dividir de manera eficiente un conjunto de puntos en el espacio n-dimensional. Esta deriva del Ham Sandwich Theorem, por lo que es bastante importante entenderlo primeramente.

## Ham Sandwich Theorem
El teorema del pan con jamón dice que teniendo n objetos en el espacio, existe un único hiperplano n-1 dimensional que corta simultáneamente todos estos n objetos en dos partes de igual volumen (o medida). El nombre deriva del caso n = 3, en el que si tienes dos pedazos de pan y uno de jamón en un espacio tridimensional entonces existe un único plano que corta cada uno de los tres objetos en dos particiones de igual volumen.

Este teorema cuando trabajas con puntos en vez de volúmenes te dice que siempre hay una partición en la cual divides los puntos en el plano en cantidades iguales.

### Demostración n = 2 (El teorema del pancake)
Se procede a hacer una demostración para el caso específico n = 2: $\newline$
Sean dos regiones disjuntas a dividir $K_1$ y $K_2$ $\newline$
Sea el círculo $S^1 = \{(x,y) \in \mathbb{R}^2 : x^2 + y^2 = 1\}$, cada dirección en un plano tiene un vector unitario correspondiente, este puede ser pensado como un punto en el círculo $S^1$. Para cada dirección $m = (\cos(\theta), \sin(\theta)) \in S^1$, hay una única línea l(m) con pendiente $\tan(\theta)$ que divide la región $K_1$ en partes igual área.

Además, la línea l(m) divide $K_2$ en dos regiones, vamos a llamar a una de ella la parte positiva $P(m)$ y a la otra la parte negativa $N(m)$. Luego definimos una función $f: S^1 \to \mathbb{R}$ tal que

$$f(m) := Área(K_2 \cap P(m))$$

Es decir, f(m) es el área contenida en la parte positiva de l(m). Dado que f(m) es una función continua, se puede usar el teorema de Borsuk-Ulman, el cual es un corolario del teorema del valor intermedio. Este teorema afirma que si $f: S^n \to \mathbb{R}^n$ es continua, entonces existe $x \in S^n$ tal que: $f(-x) = f(x)$

# TODO

# HSTNode class

In [1]:
from typing import List, Optional
import numpy as np

class HSTNode:
    def __init__(self):
        self.root: Optional['HSTNode'] = None
        self.height: int = 0.0

        self.points: List[np.ndarray] = []  

        self.medianProj: float
        self.direction: np.ndarray = np.zeros(3)  
        self.medianPoint: np.ndarray = np.zeros(3)

        self.parent: Optional['HSTNode'] = None

        self.left: Optional['HSTNode'] = None
        self.right: Optional['HSTNode'] = None


# HST class

In [3]:
from typing import List, Optional
import numpy as np
import random
import math


class HamSandwichTree:
    def __init__(self, is3D: bool, quantity: int = 100, boundary: float = 10):
        self.is3D = is3D    
        self.quantity = quantity
        self.boundary = boundary
        self.points: List[np.ndarray] = [] 
        self.root: Optional[HSTNode] = None
    #main
    def Start(self):
        random.seed()
        self.SpawnRandomPoints(self.quantity, self.boundary)
        self.CreateTree()

    def CreateTree(self):
        self.root = self.CreateNode()
        self.root.points = self.points
        self.CreateTreeAux(self.root)

    def CreateTreeAux(self, node: HSTNode):
        if len(node.points) < 5:
            return
        self.SplitPoints(node)
        self.CreateTreeAux(node.left)
        self.CreateTreeAux(node.right)

    def CreateNode(self, parent: Optional[HSTNode] = None) -> HSTNode:
        newNode = HSTNode()
        newNode.root = self.root
        newNode.parent = parent
        if parent is not None:
            newNode.height = parent.height + 1
        return newNode

    def SpawnRandomPoints(self, quantity: int, boundary: float):
        for i in range(quantity):
            x = random.uniform(-boundary, boundary)
            y = random.uniform(-boundary, boundary)
            z = 0.0
            if self.is3D:
                z = random.uniform(-boundary, boundary)
                
            pos = np.array([x, y, z], dtype=float)
            self.points.append(pos)

    # HAM SANDWICH ALGORITHM 
    def SplitPoints(self, node: HSTNode):
        theta = random.uniform(0, math.pi)
        phi = random.uniform(0, math.pi)

        if self.is3D:
            direction = np.array([
                math.cos(theta) * math.sin(phi),
                math.sin(theta) * math.sin(phi),
                math.cos(phi)
            ], dtype=float)
        else:
            direction = np.array([
                math.cos(theta),
                math.sin(theta),
                0.0
            ], dtype=float)

        projections = [{'point': p, 'projection': np.dot(p, direction)} for p in node.points]
        projections.sort(key=lambda e: e['projection'])

        median_index = len(projections) // 2
        medianPoint = projections[median_index]['point']
        medianProj = projections[median_index]['projection']

        rightPoints: List[np.ndarray] = []
        leftPoints: List[np.ndarray] = []

        for element in projections:
            if element['projection'] >= medianProj:
                rightPoints.append(element['point'])
            else:
                leftPoints.append(element['point'])

        node.left = self.CreateNode(node)
        node.right = self.CreateNode(node)
        node.left.points = leftPoints
        node.right.points = rightPoints
        node.direction = direction
        node.medianPoint = medianPoint
        node.medianProj = medianProj



# Profiling


In [ ]:
import cProfile
import pstats
import random
import unittest
from typing import List, Tuple
import numpy as np
import os


if not os.path.exists('data'):
    os.makedirs('data', exist_ok=True)

class HSTProfile(unittest.TestCase):
    Quantities = [100, 1000, 10000]  
    Runs = 1000                       
    OutputFileName = 'data/stats_hst.csv'
    OutputFileNameSplit = 'data/stats_hst_split.csv'

    @staticmethod
    def write_header(f) -> None:
        f.write('test_case,quantity,method_name,total_time,cumulative_time,per_call_time\n')

    @staticmethod
    def write_row(f, test_case: str, quantity: int, method_name: str, total_time: float,
                   cumulative_time: float, per_call_time: float) -> None:
        f.write(f'{test_case},{quantity},{method_name},{total_time},{cumulative_time},{per_call_time}\n')

    @staticmethod
    def get_running_times(st: pstats.Stats, method_name: str) -> List[Tuple[str, float, float, float]]:
        ps = st.strip_dirs().stats

        def is_method(k):
            return method_name in k[2]

        keys = list(filter(is_method, ps.keys()))
        return [(key[2], ps[key][2], ps[key][3], ps[key][3] / ps[key][1]) for key in keys]

    def test_profile_start_method(self) -> None:
        with open(HSTProfile.OutputFileName, 'w') as f:
            HSTProfile.write_header(f)
            for qty in HSTProfile.Quantities:
                for _ in range(HSTProfile.Runs):
                    tree = HamSandwichTree(is3D=True, quantity=qty, boundary=10)
                    pro = cProfile.Profile()
                    pro.runcall(tree.Start)
                    st = pstats.Stats(pro)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HSTProfile.get_running_times(st, 'Start'):
                        HSTProfile.write_row(f, 'HST', qty, method_name, total_time, cumulative_time, per_call_time)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HSTProfile.get_running_times(st, 'CreateTree'):
                        HSTProfile.write_row(f, 'HST', qty, method_name, total_time, cumulative_time, per_call_time)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HSTProfile.get_running_times(st, 'CreateTreeAux'):
                        HSTProfile.write_row(f, 'HST', qty, method_name, total_time, cumulative_time, per_call_time)

    def test_profile_split_method(self) -> None:
        with open(HSTProfile.OutputFileNameSplit, 'w') as f:
            HSTProfile.write_header(f)
            for qty in HSTProfile.Quantities:
                for _ in range(HSTProfile.Runs):
                    tree = HamSandwichTree(is3D=True, quantity=qty, boundary=10)
                    tree.SpawnRandomPoints(tree.quantity, tree.boundary)
                    root = tree.CreateNode()
                    root.points = tree.points
                    # Queremos medir SplitPoints explícitamente
                    pro = cProfile.Profile()
                    pro.runcall(tree.SplitPoints, root)
                    st = pstats.Stats(pro)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HSTProfile.get_running_times(st, 'SplitPoints'):
                        HSTProfile.write_row(f, 'HST', qty, method_name, total_time, cumulative_time, per_call_time)

    def test_profile_mixed_operations(self) -> None:
        with open('data/stats_hst_mixed.csv', 'w') as f:
            HSTProfile.write_header(f)
            for qty in HSTProfile.Quantities:
                for _ in range(HSTProfile.Runs):
                    tree = HamSandwichTree(is3D=True, quantity=qty, boundary=10)
                    pro = cProfile.Profile()
                    pro.runcall(tree.Start)
                    st = pstats.Stats(pro)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HSTProfile.get_running_times(st, 'Start'):
                        HSTProfile.write_row(f, 'HST', qty, method_name, total_time, cumulative_time, per_call_time)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HSTProfile.get_running_times(st, 'SplitPoints'):
                        HSTProfile.write_row(f, 'HST', qty, method_name, total_time, cumulative_time, per_call_time)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HSTProfile.get_running_times(st, 'CreateTree'):
                        HSTProfile.write_row(f, 'HST', qty, method_name, total_time, cumulative_time, per_call_time)


# Ejecuta la suite de tests.
suite = unittest.TestLoader().loadTestsFromTestCase(HSTProfile)
unittest.TextTestRunner(verbosity=2).run(suite)
